In [ ]:
!pip install transformers datasets evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 71.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 KB 46.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 100.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 21.0 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
from datasets import load_dataset

dataset = load_dataset("minoosh/IEMOCAP_Text")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/10039 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/minoosh___parquet/minoosh--IEMOCAP_Text-1dc4c5d7b2f009c7/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["transcription"], truncation=True)

In [ ]:
tokenized_dataset = dataset.map(preprocess_function, batched=True)

  0%|          | 0/11 [00:00<?, ?ba/s]

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
import evaluate

accuracy = evaluate.load("accuracy")

In [ ]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
dataset = dataset.rename_column("emotion", "label")

In [ ]:
labels = dataset["train"].features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label

id2label["1"]

'hap'

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-uncased", num_labels=11, id2label=id2label, label2id=label2id
)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
tokenized_dataset = tokenized_dataset.rename_column("emotion", "label")

In [ ]:
splited_tokenized_dataset = tokenized_dataset['train'].train_test_split(test_size=0.2)
splited_tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['transcription', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 8031
    })
    test: Dataset({
        features: ['transcription', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2008
    })
})

In [ ]:
batch_size = 16

training_args = TrainingArguments(
    output_dir="finetuned_bert-base-uncased",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=20,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=splited_tokenized_dataset["train"],
    eval_dataset=splited_tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

Cloning https://huggingface.co/minoosh/finetuned_bert-base-uncased into local empty directory.


Download file pytorch_model.bin:   0%|          | 16.4k/418M [00:00<?, ?B/s]

Download file runs/Feb08_10-43-45_18b2d2006935/events.out.tfevents.1675853140.18b2d2006935.276.0:  45%|####5  …

Download file runs/Feb08_12-04-49_f873344109b0/1675857989.3221283/events.out.tfevents.1675857989.f873344109b0.…

Download file training_args.bin: 100%|##########| 3.43k/3.43k [00:00<?, ?B/s]

...

Clean file runs/Feb08_10-43-45_18b2d2006935/1675853140.1672206/events.out.tfevents.1675853140.18b2d2006935.276…

Clean file runs/Feb08_10-12-51_f339a4904974/1675851181.5398304/events.out.tfevents.1675851181.f339a4904974.584…

Clean file runs/Feb08_10-43-45_18b2d2006935/events.out.tfevents.1675853140.18b2d2006935.276.0:   5%|5         …

Clean file runs/Feb08_12-04-49_f873344109b0/events.out.tfevents.1675857989.f873344109b0.656.0:   5%|4         …

Clean file pytorch_model.bin:   0%|          | 1.00k/418M [00:00<?, ?B/s]

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: transcription. If transcription are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 8031
  Num Epochs = 20
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 10040
  Number of trainable parameters = 109490699
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faste

Epoch,Training Loss,Validation Loss,Accuracy
1,1.736500,1.516699,0.428785
2,1.349500,1.479670,0.459163
3,1.113100,1.509287,0.452689
4,0.921300,1.650115,0.452191
5,0.778700,1.749420,0.440737
6,0.659400,1.859982,0.441733
7,0.580700,1.997389,0.441235
8,0.514200,2.088660,0.427291
9,0.471600,2.155594,0.427291
10,0.436400,2.284665,0.434761


The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: transcription. If transcription are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2008
  Batch size = 16
Saving model checkpoint to finetuned_bert-base-uncased/checkpoint-502
Configuration saved in finetuned_bert-base-uncased/checkpoint-502/config.json
Model weights saved in finetuned_bert-base-uncased/checkpoint-502/pytorch_model.bin
tokenizer config file saved in finetuned_bert-base-uncased/checkpoint-502/tokenizer_config.json
Special tokens file saved in finetuned_bert-base-uncased/checkpoint-502/special_tokens_map.json
tokenizer config file saved in finetuned_bert-base-uncased/tokenizer_config.json
Special tokens file saved in finetuned_bert-base-uncased/special_tokens_map.json
The following columns in the evaluation set don't have a correspond

TrainOutput(global_step=10040, training_loss=0.5865391175585439, metrics={'train_runtime': 2730.2996, 'train_samples_per_second': 58.829, 'train_steps_per_second': 3.677, 'total_flos': 4046094667801182.0, 'train_loss': 0.5865391175585439, 'epoch': 20.0})

In [ ]:
trainer.push_to_hub()

Saving model checkpoint to finetuned_bert-base-uncased
Configuration saved in finetuned_bert-base-uncased/config.json
Model weights saved in finetuned_bert-base-uncased/pytorch_model.bin
tokenizer config file saved in finetuned_bert-base-uncased/tokenizer_config.json
Special tokens file saved in finetuned_bert-base-uncased/special_tokens_map.json
Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 32.0k/418M [00:00<?, ?B/s]

Upload file runs/Feb08_14-15-00_144a3875a8da/events.out.tfevents.1675865810.144a3875a8da.224.0: 100%|#########…

remote: Scanning LFS files for validity...        
remote: LFS file scan complete.        
To https://huggingface.co/minoosh/finetuned_bert-base-uncased
   23f140f..2e98318  main -> main

remote: LFS file scan complete.        
To https://huggingface.co/minoosh/finetuned_bert-base-uncased
   23f140f..2e98318  main -> main

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Text Classification', 'type': 'text-classification'}, 'metrics': [{'name': 'Accuracy', 'type': 'accuracy', 'value': 0.4262948207171315}]}
To https://huggingface.co/minoosh/finetuned_bert-base-uncased
   2e98318..fa03fb9  main -> main

   2e98318..fa03fb9  main -> main



'https://huggingface.co/minoosh/finetuned_bert-base-uncased/commit/2e983186600302ee1ba7e7286ea9e5fb0e8b266c'

In [ ]:
############
#Inference

In [ ]:
#text = "This was a masterpiece. Not completely faithful to the books, but enthralling from beginning to end. Might be my favorite of the three."

In [ ]:
'''from transformers import pipeline

classifier = pipeline(model="minoosh/my_first_model")
classifier(text)'''

'from transformers import pipeline\n\nclassifier = pipeline(model="minoosh/my_first_model")\nclassifier(text)'